<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/rocket_league_data_get.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# rocket league ds BUT THE DATA ONLY
i'm plat in game but gc in data science

# load in

In [ ]:
!pip install --upgrade protobuf==3.6.1
!pip install --upgrade folium==0.8.3
!pip install --upgrade imgaug==0.2.9
!pip install carball --quiet
!pip install shap --quiet
!pip install --upgrade pandas==0.24.2
!pip install pandas-datareader --upgrade --quiet

Requirement already up-to-date: protobuf==3.6.1 in /usr/local/lib/python3.6/dist-packages (3.6.1)
Requirement already up-to-date: folium==0.8.3 in /usr/local/lib/python3.6/dist-packages (0.8.3)
Requirement already up-to-date: imgaug==0.2.9 in /usr/local/lib/python3.6/dist-packages (0.2.9)
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.0.3 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 1.0.3 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/19/74/e50234bc82c553fecdbd566d8650801e3fe2d6d8c8d940638e3d8a7c5522/pandas-0.24.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.24.2 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.24.2 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.24.2 which is

In [ ]:
import warnings
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error, confusion_matrix
import itertools
import random
import shap
import carball
import os
import gzip
from carball.json_parser.game import Game
from carball.analysis.analysis_manager import AnalysisManager
import matplotlib.animation as animation
from matplotlib import rc

import codecs
decode_hex = codecs.getdecoder("hex_codec")

rc('animation', html='html5')
sns.set_context('talk')


In [ ]:
def load_rl_replay(rl_replay_file):
  _json = carball.decompile_replay(rl_replay_file)

  game = Game()
  game.initialize(loaded_json=_json)
  analysis_manager = AnalysisManager(game)
  analysis_manager.create_analysis()

  # write proto out to a file
  # read api/*.proto for info on the object properties
  with open(os.path.join('output.pts'), 'wb') as fo:
      analysis_manager.write_proto_out_to_file(fo)

  # write pandas dataframe out as a gzipped numpy array
  with gzip.open(os.path.join('output.gzip'), 'wb') as fo:
      analysis_manager.write_pandas_out_to_file(fo)

  # return the proto object in python
  proto_object = analysis_manager.get_protobuf_data()

  # return the pandas data frame in python
  dataframe = analysis_manager.get_data_frame()
  return dataframe, game, proto_object

def get_df_players(game):
  df_players = pd.DataFrame()
  for x in game.players:
    df_players = (df_players
                  .append(pd.DataFrame([[int(x.online_id),x.name,x.team.name,x.is_orange]],
                                      columns=['id','name','team','is_orange']),
                          ignore_index=True))
  return df_players

def get_game_info(game):
  warnings.filterwarnings("ignore")
  df_players = get_df_players(game)
  game_info = pd.DataFrame([[format(game.datetime,"%Y%m%d%H%M%S"),game.game_info.match_guid,int(game.name[4]),game.id]],columns=['match_date','match_id','game_number','game_id'])
  game_info = pd.concat([game_info,df_players],axis=1)
  game_info = game_info.fillna(method='ffill')
  return game_info

def get_df_game(dataframe,game_info):
  warnings.filterwarnings("ignore")
  game_info_cols = ['match_date', 'match_id', 'game_number', 'game_id']
  _game_info = game_info.loc[[1],game_info_cols].drop_duplicates()

  df_game = dataframe['game']
  df_game['goal_number'] = df_game['goal_number'].fillna(-np.inf)
  df_game['seconds'] = df_game['seconds_remaining'].sub(300).abs()
  df_game = pd.concat([_game_info,df_game],axis=1)
  df_game = df_game.reset_index().rename(columns={'index':'frames'})
  df_game.loc[:,game_info_cols] = df_game.loc[:,game_info_cols].fillna(method='ffill')
  return df_game

def get_df_hits(proto_object,game):
  name_map = {player.name: int(player.online_id) for player in game.players}
  _id2name = pd.DataFrame.from_dict([name_map]).T.reset_index().rename(columns={'index':'player_name',0:'player_id'})
  hits = proto_object.game_stats.hits
  df_hits = pd.DataFrame()
  for hit in hits:
    _df = pd.DataFrame([[proto_object.game_metadata.match_guid,proto_object.game_metadata.id,hit.goal,hit.frame_number,int(hit.player_id.id),hit.collision_distance,hit.ball_data.pos_x,hit.ball_data.pos_y,hit.ball_data.pos_z,hit.distance,hit.distance_to_goal,hit.next_hit_frame_number,hit.goal_number,hit.is_kickoff]],columns=['match_id','game_id','goal','frame_number','player_id','collision_distance','pos_x','pos_y','pos_z','distance','distance_to_goal','next_hit_frame_number','goal_number','is_kickoff'])
    df_hits = df_hits.append(_df,ignore_index=True)
  df_hits = df_hits.merge(_id2name)
  return df_hits

def flatten_dataframe(dataframe,game_info):
  warnings.filterwarnings("ignore")
  game_info_cols = ['match_date','match_id','game_number','game_id','id','name','team','is_orange']
  to_melt = [x for x in dataframe.columns.levels[0].values if x not in ['ball', 'game']]

  flat_df = pd.DataFrame()
  for player_name in to_melt:
    _df = dataframe[player_name]
    _df = _df.reset_index().rename(columns={'index':'frames'})
    column_list = _df.columns.values
    column_list = game_info_cols+column_list.tolist()
    _df = pd.concat([game_info.loc[game_info['name']==player_name],_df],axis=1)
    _df.loc[:,game_info_cols] = _df.loc[:,game_info_cols].fillna(method='bfill').fillna(method='ffill')
    flat_df = flat_df.append(_df,ignore_index=True)
  flat_df = flat_df.loc[:,column_list]
  flat_df = generate_zones(flat_df)
  return flat_df

def generate_zones(df):
  zone_markers = {'x': 1100.0, 'y': 1750.0}
  for col in ['x', 'y', 'z']:
    df[f'{col}_zone'] = 0


  df.loc[df['pos_x']<-zone_markers['x'],'x_zone'] = -1
  df.loc[df['pos_x'].between(-zone_markers['x'],zone_markers['x']),'x_zone'] = 0
  df.loc[df['pos_x']>zone_markers['x'],'x_zone'] = 1

  df.loc[df['pos_y']<-zone_markers['y'],'y_zone'] = -1
  df.loc[df['pos_y'].between(-zone_markers['y'],zone_markers['y']),'y_zone'] = 0
  df.loc[df['pos_y']>zone_markers['y'],'y_zone'] = 1

  df.loc[df['pos_z']<500,'z_zone'] = 0
  df.loc[df['pos_z'].between(500,1500),'z_zone'] = 1
  df.loc[df['pos_z']>1500,'z_zone'] = 2
  return df

def get_df_ball(dataframe,game_info):
  warnings.filterwarnings("ignore")
  game_info_cols = ['match_date', 'match_id', 'game_number', 'game_id']
  _game_info = game_info.loc[[1],game_info_cols].drop_duplicates()
  df_ball = dataframe['ball']
  df_ball = generate_zones(df_ball)
  df_ball = pd.concat([_game_info,df_ball],axis=1)
  df_ball = df_ball.reset_index().rename(columns={'index':'frames'})
  df_ball.loc[:,game_info_cols] = df_ball.loc[:,game_info_cols].fillna(method='ffill')
  return df_ball

def get_df_goals(game,game_info):
  warnings.filterwarnings("ignore")
  game_info_cols = ['match_date', 'match_id', 'game_number', 'game_id']
  _game_info = game_info.loc[[1],game_info_cols].drop_duplicates()
  df_goals = pd.DataFrame()
  for idx_goal in range(len(game.goals)):
    goal = game.goals[idx_goal]
    df_goal = pd.DataFrame([[int(goal.player.online_id),goal.player.name,bool(goal.player_team),goal.frame_number]], columns=['id','name','is_orange','goal_frame_number'])
    df_goals = df_goals.append(df_goal,ignore_index=True)
    df_goals = df_goals.sort_values('goal_frame_number').reset_index(drop=True)
  df_goals = pd.concat([_game_info,df_goals],axis=1)
  df_goals.loc[:,game_info_cols] = df_goals.loc[:,game_info_cols].fillna(method='ffill').fillna(method='bfill')
  return df_goals

def plot_gridlines(view='xy'):
  if view=='xy':
    plt.axvline(-4000,c='k')
    plt.axvline(4000,c='k')
    plt.axvline(1333,c='r',ls=':')
    plt.axvline(-1333,c='r',ls=':')

    plt.axhline(-6000,c='k')
    plt.axhline(6000,c='k')
    plt.axhline(-2000,c='r',ls=':')
    plt.axhline(2000,c='r',ls=':')

  elif view=='xz':
    plt.axvline(-4000,c='k')
    plt.axvline(4000,c='k')
    plt.axvline(1333,c='r',ls=':')
    plt.axvline(-1333,c='r',ls=':')

    plt.axhline(0,c='k')
    plt.axhline(2000,c='k')
    plt.axhline(500,c='r',ls=':')
    plt.axhline(1500,c='r',ls=':')

  elif view=='yz':
    plt.axvline(-6000,c='k')
    plt.axvline(6000,c='k')
    plt.axvline(-2000,c='r',ls=':')
    plt.axvline(2000,c='r',ls=':')

    plt.axhline(0,c='k')
    plt.axhline(2000,c='k')
    plt.axhline(500,c='r',ls=':')
    plt.axhline(1500,c='r',ls=':')

  sns.despine(left=True,bottom=True)

def make_positional_heatmap(player_name,sample=15000,zone_type='xy'):
  warnings.filterwarnings("ignore")
  df_player = flat_df.loc[flat_df['name']==player_name].sample(sample)

  if zone_type == 'xy':
    axis_1 = 'x'
    axis_2 = 'y'
    index = {-1:'atk',0:'mid',1:'def'}
    columns = {-1:'left',0:'center',1:'right'}
    df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'] = df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'].mul(-1)
    df_player.loc[df_player['is_orange']==0,f'{axis_2}_zone'] = df_player.loc[df_player['is_orange']==0,f'{axis_2}_zone'].mul(-1)

  elif zone_type == 'xz':
    axis_1 = 'x'
    axis_2 = 'z'
    columns = {-1:'left',0:'center',1:'right'}
    index = {0:'floor',-1:'air',-2:'ceiling'}
    df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'] = df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'].mul(-1)
    df_player.loc[:,f'{axis_2}_zone'] = df_player.loc[:,f'{axis_2}_zone'].mul(-1)

  elif zone_type == 'yz':
    axis_1 = 'y'
    axis_2 = 'z'
    columns = {-1:'atk',0:'mid',1:'def'}
    index = {0:'floor',-1:'air',-2:'ceiling'}
    df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'] = df_player.loc[df_player['is_orange']==0,f'{axis_1}_zone'].mul(-1)
    df_player.loc[:,f'{axis_2}_zone'] = df_player.loc[:,f'{axis_2}_zone'].mul(-1)

  df_position = df_player.loc[:,[f'{axis_1}_zone',f'{axis_2}_zone','frames']].groupby([f'{axis_1}_zone',f'{axis_2}_zone'],as_index=False).sum()
  df_position['frame_pct'] = df_position['frames'].div(df_position['frames'].sum())
  _df = pd.pivot_table(df_position.loc[:,[f'{axis_1}_zone',f'{axis_2}_zone','frame_pct']],
                values=['frame_pct'],columns=f'{axis_1}_zone',index=f'{axis_2}_zone')
  _df = _df.rename(index=index,columns=columns)
  _df = _df['frame_pct']

  avg_heatmap = _get_avg_heatmap(flat_df,zone_type=zone_type)
  _df = (_df.sub(avg_heatmap)).div(avg_heatmap)

  f = sns.heatmap(_df,vmin=-0.5,vmax=0.5,cmap='coolwarm',square=True)
  b, t = f.get_ylim() # discover the values for bottom and top
  b += 0.5 # Add 0.5 to the bottom
  t -= 0.5 # Subtract 0.5 from the top
  f.set_ylim(b, t) # update the ylim(bottom, top) values
  plt.title(f"{player_name}")
  plt.show()

def calc_magnitude(df):
  magnitude = np.sqrt(df['vel_x']**2+df['vel_y']**2+df['vel_z']**2)
  return magnitude

def touches2modeldata(tid,touches):
  stat_cols = ['goals_per_game', 'hits_per_game', 'goals_per_hit',
              'x_zone', 'y_zone', 'z_zone', 'x_zone_pct',
              'y_zone_pct', 'z_zone_pct']

  _team = touches.loc[touches['team']==tid]
  id_shuffle = _team['id'].unique().tolist()
  random.shuffle(id_shuffle)
  for idx,pid in enumerate(id_shuffle):
    rename_stat_cols = {k:f"{k}_{idx}" for k in stat_cols}
    _player = _team.loc[_team['id']==pid,stat_cols].rename(columns=rename_stat_cols)
    if idx == 0:
      team = _player
    else:
      team = pd.concat([team.reset_index(drop=True),_player.reset_index(drop=True)],axis=1)

  return team

def matchup_win_prob(model_data,touches,team1,team2='BULL SEND'):
  model_feats = ['goals_per_game_0_0', 'goals_per_game_1_0',
        'goals_per_game_2_0', 'goals_per_hit_0_0', 'goals_per_hit_1_0',
        'goals_per_hit_2_0', 'hits_per_game_0_0', 'hits_per_game_1_0',
        'hits_per_game_2_0', 'x_zone_0_0', 'x_zone_1_0',
        'x_zone_2_0', 'x_zone_pct_0_0', 'x_zone_pct_1_0', 'x_zone_pct_2_0',
        'y_zone_0_0', 'y_zone_1_0', 'y_zone_2_0', 'y_zone_pct_0_0',
        'y_zone_pct_1_0', 'y_zone_pct_2_0', 'z_zone_0_0', 'z_zone_1_0',
        'z_zone_2_0', 'z_zone_pct_0_0', 'z_zone_pct_1_0', 'z_zone_pct_2_0',
        'goals_per_game_0_1', 'goals_per_game_1_1', 'goals_per_game_2_1',
        'goals_per_hit_0_1', 'goals_per_hit_1_1', 'goals_per_hit_2_1',
        'hits_per_game_0_1', 'hits_per_game_1_1', 'hits_per_game_2_1',
        'x_zone_0_1', 'x_zone_1_1', 'x_zone_2_1',
        'x_zone_pct_0_1', 'x_zone_pct_1_1', 'x_zone_pct_2_1', 'y_zone_0_1',
        'y_zone_1_1', 'y_zone_2_1', 'y_zone_pct_0_1', 'y_zone_pct_1_1',
        'y_zone_pct_2_1', 'z_zone_0_1', 'z_zone_1_1', 'z_zone_2_1',
        'z_zone_pct_0_1', 'z_zone_pct_1_1', 'z_zone_pct_2_1']

  target = 'win'

  win_prob_data = get_win_prob_data(team1, team2,touches)
  win_prob = get_win_probability(team1, team2,model_data,win_prob_data)
  return win_prob[0][1]

def get_win_probability(opp_id,bullsend_id,model_data,win_prob_data):
  train_data = model_data.loc[(~model_data['team_0'].isin([opp_id,bullsend_id]))&(~model_data['team_1'].isin([opp_id,bullsend_id]))]
  matchup_model = LogisticRegression()
  matchup_model.fit(train_data.loc[:,model_feats],train_data.loc[:,target])
  win_prob = matchup_model.predict_proba(win_prob_data.loc[:,model_feats])
  return win_prob

def get_increments(model_data):
  dist_cols = [x for x in model_data.columns.values if x[-3:]=='0_0']
  increments = {}
  for col in dist_cols:
    pct_diff = [model_data[col].drop_duplicates().quantile(x)-model_data[col].drop_duplicates().quantile(y) for y,x in zip(np.arange(0.01,1.00,0.01),np.arange(0.02,1.01,0.01))]
    increments[col[:-4]] = np.median(pct_diff)
  return increments

def get_win_prob_data(team1, team2,touches):
  _opp = touches2modeldata(team1,touches)
  _bull = touches2modeldata(team2,touches)
  win_prob_data = _opp.merge(_bull,left_index=True, right_index=True,suffixes=('_0','_1')).dropna()
  return win_prob_data

def data_tuning(win_prob_data, increments, col, improve=True):
  df = win_prob_data.copy()
  if improve:
    team_str = '1'
  else:
    team_str = '0'
  cols_to_tune = [y for y in [x for x in win_prob_data.columns.values if x[:-4] in col] if y[-1] == team_str]
  rand_col = random.choice(cols_to_tune)
  df.loc[:,rand_col] = win_prob_data.loc[:,rand_col].add(increments[col])
  return df

def plot_rl_field_aesthetics(fig1, ax):
  """
  note: probably want to parameterize the field dims (for smaller field/"extra" game modes)

  """

  boost_locs = pd.DataFrame([[-3072.0, -4096.0, 73.0, 'big'],
                           [ 3072.0, -4096.0, 73.0, 'big'],
                           [-3584.0,     0.0, 73.0, 'big'],
                           [ 3584.0,     0.0, 73.0, 'big'],
                           [-3072.0,  4096.0, 73.0, 'big'],
                           [ 3072.0,  4096.0, 73.0, 'big'],
                           [    0.0, -4240.0, 70.0, 'small'],
                           [-1792.0, -4184.0, 70.0, 'small'],
                           [ 1792.0, -4184.0, 70.0, 'small'],
                           [- 940.0, -3308.0, 70.0, 'small'],
                           [  940.0, -3308.0, 70.0, 'small'],
                           [    0.0, -2816.0, 70.0, 'small'],
                           [-3584.0, -2484.0, 70.0, 'small'],
                           [ 3584.0, -2484.0, 70.0, 'small'],
                           [-1788.0, -2300.0, 70.0, 'small'],
                           [ 1788.0, -2300.0, 70.0, 'small'],
                           [-2048.0, -1036.0, 70.0, 'small'],
                           [    0.0, -1024.0, 70.0, 'small'],
                           [ 2048.0, -1036.0, 70.0, 'small'],
                           [-1024.0,     0.0, 70.0, 'small'],
                           [ 1024.0,     0.0, 70.0, 'small'],
                           [-2048.0,  1036.0, 70.0, 'small'],
                           [    0.0,  1024.0, 70.0, 'small'],
                           [ 2048.0,  1036.0, 70.0, 'small'],
                           [-1788.0,  2300.0, 70.0, 'small'],
                           [ 1788.0,  2300.0, 70.0, 'small'],
                           [-3584.0,  2484.0, 70.0, 'small'],
                           [ 3584.0,  2484.0, 70.0, 'small'],
                           [    0.0,  2816.0, 70.0, 'small'],
                           [- 940.0,  3310.0, 70.0, 'small'],
                           [  940.0,  3308.0, 70.0, 'small'],
                           [-1792.0,  4184.0, 70.0, 'small'],
                           [ 1792.0,  4184.0, 70.0, 'small'],
                           [    0.0,  4240.0, 70.0, 'small']],
                          columns=['boost_pad_x', 'boost_pad_y',
                                   'boost_pad_z', 'boost_pad_type'])
  boost_locs['group']=1

  fig1 = sns.lineplot([-4096,-4096+1152], [-5120+1152, -5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([4096,4096-1152], [5120-1152, 5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([-4096,-4096+1152], [5120-1152, 5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([4096,4096-1152], [-5120+1152, -5120], color='k', linestyle='-', ax=ax, alpha=1)

  fig1 = sns.lineplot([-4096,-4096], [5120-1152, -5120+1152], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([4096,4096], [5120-1152, -5120+1152], color='k', linestyle='-', ax=ax, alpha=1)

  fig1 = sns.lineplot([-4096+1152,-893], [5120, 5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([4096-1152,893], [5120, 5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([-893,893], [5120, 5120], color='k', linestyle=':', ax=ax, alpha=1)

  fig1 = sns.lineplot([-4096+1152,-893], [-5120, -5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([4096-1152,893], [-5120, -5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([-893,893], [-5120, -5120], color='k', linestyle=':', ax=ax, alpha=1)

  fig1 = sns.lineplot([-893,893], [5120+880, 5120+880], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([893,893], [5120+880, 5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([-893,-893], [5120+880, 5120], color='k', linestyle='-', ax=ax, alpha=1)

  fig1 = sns.lineplot([-893,893], [-5120-880, -5120-880], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([893,893], [-5120-880, -5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1 = sns.lineplot([-893,-893], [-5120-880, -5120], color='k', linestyle='-', ax=ax, alpha=1)
  fig1.axhline(0, c='k', ls='--', lw=1)
  fig1 = sns.scatterplot(x='boost_pad_x', y='boost_pad_y',data=boost_locs, hue='group', palette=[(60/255, 60/255, 60/255)], alpha=.6, size='boost_pad_type', sizes=(5,15), ax=ax)

def get_play_by_frame(fid, ax, df, cmaps):
  ax.cla()
  _teams = df.loc[:, ['team', 'is_orange']].drop_duplicates()
  hue_order = [_teams.loc[_teams['is_orange']==0, 'team'].values[0], _teams.loc[_teams['is_orange']==1, 'team'].values[0], 'RL_BALL']
  gid = df['match_id'].unique()[0]
  pid = df['game_id'].unique()[0]
  one_frame = df.loc[df['frames']==fid]
  cmap = cmaps[fid]
  fig1 = sns.scatterplot(x='pos_x', y='pos_y', data=one_frame, hue='team', ax=ax, hue_order=hue_order, size='pos_z', sizes=(30,150), palette=[(0/255, 221/255, 255/255),(255/255, 200/255, 0/255), cmap])
  # fig1 = sns.scatterplot(x='pos_x', y='pos_y', data=one_frame, ax=ax, size='pos_z', sizes=(30,150), hue='team', hue_order=hue_order+['RL_BALL'], palette=[(0,0,0,0),(0,0,0,0),cmap], edgecolor='black',zorder=4)
  # fig1 = sns.scatterplot(x='pos_x', y='pos_y', data=one_frame.loc[one_frame['team']!='RL_BALL'], ax=ax, size='pos_z', sizes=(30,150))
  plot_rl_field_aesthetics(fig1, ax)

  fig1.set_title(f"BLUE: {hue_order[0]}\nORANGE: {hue_order[1]}")
  sns.despine(left=True, bottom=True)
  fig1.set_ylabel('')
  fig1.set_yticks([])
  fig1.set_xlabel('')
  fig1.set_xticks([])
  fig1.set_xlim(-4196,4196)
  fig1.set_ylim(-6100,6100)
  fig1.set_aspect('equal')
  fig1.legend([]).set_visible(False)

def animate_predicted_play(df, suffix='_pred'):
  gid = df['match_id'].unique()[0]
  pid = df['game_id'].unique()[0]
  df['frames'] = df['frames'].sub(df['frames'].min()-1)
  cmaps = df.loc[df['team']=='RL_BALL', 'cmap'].values.tolist()

  fig = plt.figure(figsize=(4,6))
  ax = fig.gca()
  ani = animation.FuncAnimation(fig, get_play_by_frame,
                                frames=df['frames'].sort_values().unique().shape[0],
                                interval=50, repeat=True, fargs=(ax,df,cmaps,))

  plt.close()
  # ani.save('test.gif', writer='imagemagick', fps=20)
  return ani

In [ ]:
root_dir = '/content/drive/My Drive/rocket_league/data'
season_fns = os.listdir(root_dir)
league_fn = f'{root_dir}/RLCS Season 9/RLCS/RLCS NA League Play'

flat_df = pd.DataFrame()
df_goals = pd.DataFrame()
df_game = pd.DataFrame()
df_ball = pd.DataFrame()
df_hits = pd.DataFrame()

week_fns = [x for x in os.listdir(league_fn) if 'Week' in x]

In [ ]:
for week_fn in week_fns:
  match_fns = os.listdir(f"{league_fn}/{week_fn}")
  for match_fn in match_fns:
    replay_files = os.listdir(f"{league_fn}/{week_fn}/{match_fn}")
    for replay_file in replay_files:
      fn = f"{league_fn}/{week_fn}/{match_fn}/{replay_file}"
      try:
        dataframe, game, proto_object = load_rl_replay(fn)
      except:
        print(f"can't load {fn}...")
        continue
      game_info = get_game_info(game)
      _df_hits = get_df_hits(proto_object,game)
      _flat_df = flatten_dataframe(dataframe,game_info)
      _df_goals = get_df_goals(game,game_info)
      _df_game = get_df_game(dataframe,game_info)
      _df_ball = get_df_ball(dataframe,game_info)
      flat_df = flat_df.append(_flat_df,ignore_index=True)
      df_goals = df_goals.append(_df_goals,ignore_index=True)
      df_game = df_game.append(_df_game,ignore_index=True)
      df_ball = df_ball.append(_df_ball,ignore_index=True)
      df_hits = df_hits.append(_df_hits,ignore_index=True)

for col in ['x_zone', 'y_zone']:
  flat_df.loc[flat_df['is_orange']==1, col] = flat_df.loc[flat_df['is_orange']==1, col].mul(-1)

df_goals['goal_frame_number_1s'] = df_goals['goal_frame_number'].sub(20)
df_goals['goal_frame_number_5s'] = df_goals['goal_frame_number'].sub(100)

Player RLCS Admin as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Admin as player has no team.
Player Shogun as player has no MatchScore.
Score is not found for player
Ignoring player: Shogun as player has no team.
Player RLCS Observer as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Observer as player has no team.
Player JohnnyBoi_i as player has no MatchScore.
Score is not found for player
Ignoring player: JohnnyBoi_i as player has no team.


can't load /content/drive/My Drive/rocket_league/data/RLCS Season 9/RLCS/RLCS NA League Play/Week 6/M1 - Cloud9 vs Susquehanna Soniqs/33E6AD6C443AE2450F45A885AF315D3C.replay...


Player RLCS Admin as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Admin as player has no team.
Player Shogun as player has no MatchScore.
Score is not found for player
Ignoring player: Shogun as player has no team.
Player JohnnyBoi_i as player has no MatchScore.
Score is not found for player
Ignoring player: JohnnyBoi_i as player has no team.
Player RLCS Observer as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Observer as player has no team.
Goal is not shot: frame 5466 by Shock
The player never hit the ball during the "carry"
Player RLCS Admin as player has no MatchScore.
Score is not found for player
Ignoring player: RLCS Admin as player has no team.
Player Shogun as player has no MatchScore.
Score is not found for player
Ignoring player: Shogun as player has no team.
Player JohnnyBoi_i as player has no MatchScore.
Score is not found for player
Ignoring player: JohnnyBoi_i as player has no team.
Player RLCS Observer as

In [ ]:
flat_df.to_csv('/content/drive/My Drive/rocket_league/data/2019_flat_df.csv', index=False)
df_goals.to_csv('/content/drive/My Drive/rocket_league/data/2019_df_goals.csv', index=False)
df_game.to_csv('/content/drive/My Drive/rocket_league/data/2019_df_game.csv', index=False)
df_ball.to_csv('/content/drive/My Drive/rocket_league/data/2019_df_ball.csv', index=False)
df_hits.to_csv('/content/drive/My Drive/rocket_league/data/2019_df_hits.csv', index=False)